In [6]:
import os
import scanpy as sc
import muon as mu

RAW_DATA_DIR = "/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/data/raw/GSE218576"
PROCESSED_DATA_DIR = "/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/data/processed/GSE218576"
sample_filtered_feature_file = "B6E12-5-1_filtered_feature_bc_matrix.h5"
sample_fragment_file = "B6E12-5-1_atac_fragments.tsv.gz"
sample_fragment_tbi_file = "B6E12-5-1_atac_fragments.tsv.gz.tbi.gz"

# Load RNA 10X (filtered_feature_bc_matrix.h5)
rna = sc.read_10x_h5(os.path.join(RAW_DATA_DIR, sample_filtered_feature_file), gex_only=True)
print(rna)

/gpfs/Home/esm5360/.conda/envs/my_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 6736 × 32285
    var: 'gene_ids', 'feature_types', 'genome', 'interval'


/gpfs/Home/esm5360/.conda/envs/my_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [7]:
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
import numpy as np
import os

# Load peak-cell pairs
df = pd.read_csv(f"{PROCESSED_DATA_DIR}/peak_cell_hits.tsv", sep="\t", header=None, names=["peak","cell"])

# Factorize peaks and cells into indices
peaks, peak_idx = np.unique(df["peak"], return_inverse=True)
cells, cell_idx = np.unique(df["cell"], return_inverse=True)

# Build sparse count matrix
counts = csr_matrix(
    (np.ones(len(df), dtype=int), (peak_idx, cell_idx)),
    shape=(len(peaks), len(cells))
)

# Save in Matrix Market format (can be loaded by Scanpy or Seurat)
from scipy.io import mmwrite
mmwrite(f"{PROCESSED_DATA_DIR}/peak_cell_matrix.mtx", counts)

# Save peak and cell names
pd.Series(peaks).to_csv(f"{PROCESSED_DATA_DIR}/peaks.txt", index=False, header=False)
pd.Series(cells).to_csv(f"{PROCESSED_DATA_DIR}/cells.txt", index=False, header=False)

print(f"[INFO] Saved peak × cell matrix with shape {counts.shape}")

[INFO] Saved peak × cell matrix with shape (108568, 363645)
